In [2]:
from fastapi import FastAPI
import pandas as pd
import ast


In [5]:
df_games = pd.read_json('./data_limpia/output_steam_games_limpia.json', lines=True)
df_reviews = pd.read_json('./data_limpia/user_reviews.json', lines=True)
df_items = pd.read_json('./data_limpia/australian_user_items_limpia.json', lines=True)

In [20]:
def developer(desarrollador: str):
    filtered_data = df_items[df_items['developer'] == desarrollador]
    
    resultado = (
        filtered_data
        .groupby('release_date')
        .agg(cantidad_items=('item_id', 'count'),
             contenido_free=('is_free', lambda x: (x == 'Yes').mean() * 100))  
    ).reset_index() 
    
    resultado_list = resultado.to_dict(orient='records')
    
    return resultado_list

dev = input("developer")
funcion1 = developer(dev)
print(funcion1)


KeyError: 'developer'

In [21]:

def developer(desarrollador: str, games_df):
    """
    Función para analizar los juegos desarrollados por un desarrollador específico.
    
    Args:
        desarrollador (str): Nombre del desarrollador.
        games_df (DataFrame): DataFrame que contiene información de los juegos.
        
    Returns:
        DataFrame: DataFrame con el conteo de juegos lanzados por año y el porcentaje de juegos gratuitos.
    """
    
    # Convertimos 'release_date' a tipo fecha
    games_df['release_date'] = pd.to_datetime(games_df['release_date'], errors='coerce')
    
    # Filtramos por el desarrollador especificado
    df_dev = games_df[games_df['developer'] == desarrollador]

    # Agrupamos por año y contamos el número de elementos
    items_por_año = df_dev.groupby(df_dev['release_date'].dt.year).size().reset_index(name='Items')
    
    # Contamos los juegos gratuitos por año
    free_por_año = df_dev[df_dev['price'] == 0].groupby(df_dev['release_date'].dt.year).size().reset_index(name='freeitem')

    # Combinamos los resultados en un solo DataFrame
    result_df = items_por_año.merge(free_por_año, on='release_date', how='left')
    
    # Calculamos el porcentaje de juegos gratuitos
    result_df['freeitem'] = ((result_df['freeitem'].fillna(0) / result_df['Items']) * 100).round(2).astype(str) + '%'

    # Renombramos la columna 'release_date' a 'Año'
    result_df.rename(columns={'release_date': 'Año'}, inplace=True)

    return result_df

# Llamada a la función
dev = input("¿Nombre del desarrollador? ")
funcion1 = developer(dev, df_games)
print(funcion1)

    Año  Items freeitem
0  2018      1     0.0%


In [36]:
def get_user_data(User_id: str):
    df_items['user_id'] = df_items['user_id'].astype(str)
    df_reviews['user_id'] = df_reviews['user_id'].astype(str)

    user_items = df_items[df_items['user_id'] == User_id]

    df_games['price'] = pd.to_numeric(df_games['price'], errors='coerce')

    if not user_items.empty:
        money_spent = user_items.merge(df_games[['id', 'price']], left_on='item_id', right_on='id', how='left')['price'].sum()
    else:
        money_spent = 0.0

    total_items = user_items['items_count'].sum()

    user_reviews = df_reviews[df_reviews['user_id'] == User_id]

    if user_reviews.shape[0] > 0:
        recommendation_percentage = (user_reviews['recommend'].sum() / user_reviews.shape[0]) * 100
    else:
        recommendation_percentage = 0.0 

    result = {
        "Usuario": User_id,
        "Dinero gastado": f"{float(money_spent):.2f} USD",  
        "% de recomendación": f"{float(recommendation_percentage):.2f}%",
        "Cantidad de items": int(total_items)
    }

    return result


In [37]:
# Ejecutar la función
resultado = get_user_data('Wackky')  
print(resultado)

{'Usuario': 'Wackky', 'Dinero gastado': '0.00 USD', '% de recomendación': '100.00%', 'Cantidad de items': 0}
